In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.9 MB/s eta 0:00:0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data
! ls # verify that you are in the right directory

/content/drive/.shortcut-targets-by-id/1lTbeLKl99_-ux0hJXW-AtqLq-J_M1fIm/data
finalWords.json		  gpt2_predict_model
finetune_eval.txt	  gpt2_predict_model_30
finetune_train.txt	  gpt2_predict_model_6
finetune_val.txt	  predict_embeddings_model_weights
gpt2_adapter_casual	  predict_embeddings_model_weights_30
gpt2_adapter_mlm_model	  predict_embeddings_model_weights_6
gpt2_adapter_mlm_model_3  quiz_questions_10.txt
gpt2_casual		  quiz_questions_25.txt
gpt2_casual_2		  quiz_questions_2.txt
gpt2_finetune_random	  quiz_questions_50.txt
gpt2_mlm_model		  quiz_questions_5.txt
gpt2_mlm_model_2	  quiz_questions.txt
gpt2_mlm_model_3	  words.txt


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2DoubleHeadsModel.from_pretrained("gpt2")

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
####GET DATA
import torch
import json 

def get_base_word(word):
  return word.replace('Ġ', '').lower()

words = tokenizer.get_vocab()
data_process = {}
for word, index in words.items():
  b = get_base_word(word)
  # print("word", word, "base word", b, "index", index)
  if b not in data_process:
    data_process[b] = []
  data_process[b].append(model.transformer.wte.weight[index,:].detach().clone())


final_data = {}
for word in data_process:
  arr = data_process[word]
  arr = torch.stack(arr)
  final_data[word.lower()] = [word, torch.mean(arr, axis = 0)]

with open('finalWords.json', 'r') as infile:
    word_data = json.load(infile)

for i in word_data:
    word = i['word']
    word = word.lower()
    if word in final_data and len(final_data[word]) == 2:
        final_data[word].append(i["definition"])
arr = {}
output_data = {}
for i in final_data:
    l = len(final_data[i])
    if l == 3:
        output_data[final_data[i][0]] = [final_data[i][1], final_data[i][2]]
    if l not in arr:
        arr[l] = 0
    arr[l] += 1


In [ ]:
test_data = {}
new_words = [i.strip() for i in open("finetune_eval.txt").readlines()]
print(len(new_words))
words_dict = {}
for i in word_data:
  words_dict[i['word'].lower()] = i['definition']
for word in new_words:  
    test_data[word] = [words_dict[word.lower()]]
print("test_data", len(test_data))
for i in test_data:
  print(i, test_data[i])
  break

12501
test_data 12501
Salmiakki ['Salty form of licorice from Finland that is flavored with the taste of burning tires and distilled death.']


In [ ]:
from transformers import BertTokenizer

input_ids = []
attention_masks = []
x = []
y = []
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_tokenizer.pad_token = tokenizer.eos_token
words_test = []
x_test = []
input_ids_test = []
attention_masks_test = []
print("test_data", len(test_data), "output_data", len(output_data))
for i in test_data:
  words_test.append(i)
  sent = test_data[i][0]
  # print(sent)
  x_test.append(sent)
  encoded_dict = bert_tokenizer.encode_plus(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 64,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
  input_ids_test.append(encoded_dict['input_ids'])
  attention_masks_test.append(encoded_dict['attention_mask'])

for i in output_data:
  sent = output_data[i][1]
  x.append(sent)
  encoded_dict = bert_tokenizer.encode_plus(
                    sent,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 64,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
  input_ids.append(encoded_dict['input_ids'])
  attention_masks.append(encoded_dict['attention_mask'])
  y.append(output_data[i][0])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test = torch.cat(attention_masks_test, dim=0)

labels = torch.stack(y, dim=0)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


test_data 12501 output_data 21316


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
print(len(input_ids))
print(len(attention_masks))
print(len(y))
print(len(labels))

dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
# test_dataset = TensorDataset(input_ids_test, attention_masks_test, words_test)
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

21316
21316
21316
21316
19,184 training samples
2,132 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
# test_dataloader = DataLoader(
#             test_dataset, # The validation samples.
#             sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
#             batch_size = batch_size # Evaluate with this batch size.
#         )


In [ ]:
print(len(y[0]))

768


MODEL DEFINITION

In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig
import torch 
class BERT_modf(torch.nn.Module):
  def __init__(self, output_dim):
    super(BERT_modf, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    config = BertConfig.from_pretrained("bert-base-uncased")
    self.trans = torch.nn.Linear(config.hidden_size, output_dim)
  def forward(self, input_id, input_mask):
    # print("BERT_MODF FORWARD")
    bert_out = self.bert(input_id, input_mask, output_hidden_states=True)
    # print("last hidden state shape", bert_out.last_hidden_state.shape)
    bert_hidden_state = bert_out.last_hidden_state[:,0,:]
    # final_out = self.trans(bert_hidden_state)

    return self.trans(bert_hidden_state)
  
  def save(self, model_path):
    torch.save(model.state_dict(), model_path)

  def load(self, model_path):
    self.load_state_dict(torch.load(model_path))
    self.eval()

  

In [ ]:
size = len(y[0])
model = BERT_modf(size)
model = model.to(device)

print("size", size)
# params = list(model.named_parameters())

# print('The BERT model has {:} different named parameters.\n'.format(len(params)))

# print('==== Embedding Layer ====\n')

# for p in params[0:5]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# print('\n==== First Transformer ====\n')

# for p in params[5:21]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# print('\n==== Output Layer ====\n')

# for p in params[-4:]:
#     print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


size 768


In [ ]:
model.load_state_dict(torch.load("predict_embeddings_model_weights_30"))

<All keys matched successfully>

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 30

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import numpy as np

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


TRAINING LOOP

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
# seed_val = 42
MSE_loss = torch.nn.MSELoss()
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
prev_val = 1
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        predict_y = model.forward(b_input_ids, b_input_mask)
        # print("predicted", predict_y)
        # print(predict_y.shape, b_labels.shape)
        loss = MSE_loss(predict_y, b_labels)
        # loss = MSE_loss(torch.mul(torch.mean(predict_y, axis=1), 100), torch.mul(b_labels, 100))
        # loss = MSE_loss(predict_y, b_labels[:, np.newaxis, :])

        # print("loss", loss)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        # print(loss)
        total_train_loss += loss.item()
        # print(loss.item())
        # Perform a backward pass to calculate the gradients.
        # a = model.state_dict().__str__()
        loss.backward() ####freezes here
        # optimizer.step()
        # for p in model.parameters():
        #   print("gradient", p.grad)

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # b = model.state_dict().__str__()
        # print("parameters are equal", a==b)

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
        

            predict_y = model.forward(b_input_ids, b_input_mask)
            loss = MSE_loss(predict_y, b_labels)
            # loss = MSE_loss(predict_y, b_labels)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += loss.item() ##can't really compute accuracy, just make it the loss
    

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.5f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.5f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
    if avg_val_loss > prev_val:
      break
    prev_val = avg_val_loss

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

        


======== Epoch 1 / 30 ========
Training...
  Batch    40  of    600.    Elapsed: 0:00:18.
  Batch    80  of    600.    Elapsed: 0:00:31.
  Batch   120  of    600.    Elapsed: 0:00:44.
  Batch   160  of    600.    Elapsed: 0:00:57.
  Batch   200  of    600.    Elapsed: 0:01:10.
  Batch   240  of    600.    Elapsed: 0:01:24.
  Batch   280  of    600.    Elapsed: 0:01:37.
  Batch   320  of    600.    Elapsed: 0:01:50.
  Batch   360  of    600.    Elapsed: 0:02:04.
  Batch   400  of    600.    Elapsed: 0:02:17.
  Batch   440  of    600.    Elapsed: 0:02:30.
  Batch   480  of    600.    Elapsed: 0:02:44.
  Batch   520  of    600.    Elapsed: 0:02:57.
  Batch   560  of    600.    Elapsed: 0:03:10.

  Average training loss: 0.01546
  Training epcoh took: 0:03:24

Running Validation...
  Accuracy: 0.01333
  Validation Loss: 0.01333
  Validation took: 0:00:07

======== Epoch 2 / 30 ========
Training...
  Batch    40  of    600.    Elapsed: 0:00:13.
  Batch    80  of    600.    Elapsed: 0:00:27

In [ ]:
torch.save(model.state_dict(), "predict_embeddings_model_weights_30")

In [ ]:
test = []
model.eval()
print(len(words_test))
for i in range(0, len(words_test)):
  # if i % 1000 == 0:
  #   print(i)
  word = words_test[i]
  with torch.no_grad():        

    input_ids = input_ids_test[i][None, :].to(device)
    attention_mask = attention_masks_test[i][None, :].to(device)
    # print("input_ids", input_ids)
    # print("attention_mask", attention_mask)
    predict_y = model.forward(input_ids, attention_mask)
    test.append(predict_y)
print("test concluded", len(test))

12501
test concluded 12501


In [ ]:
# x is my data which is a nd-array
  # You have to convert your tensor to nd-array before using scikit-learn's tsne
  # Convert your tensor to x =====> x = tf.Session().run(tensor_x)
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, LabelEncoder

x = test.cpu().detach().numpy()
standard = StandardScaler()
x_std = standard.fit_transform(x)
plt.figure()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

tsne = TSNE(n_components=2, random_state=0)  # n_components means you mean to plot your dimensional data to 2D
x_test_2d = tsne.fit_transform(x_std)

print()

markers = ('s', 'd', 'o', '^', 'v', '8', 's', 'p', "_", '2')
color_map = {0: 'red', 1: 'blue', 2: 'lightgreen', 3: 'purple', 4: 'cyan', 5: 'black', 6: 'yellow', 7: 'magenta',
          8: 'plum', 9: 'yellowgreen'}
for idx, cl in enumerate(np.unique(y)):

    plt.scatter(x=x_test_2d[y == cl, 0], y=x_test_2d[y == cl, 1], c=color_map[idx], marker=markers[idx],
            label=cl)
plt.xlabel('X in t-SNE')
plt.ylabel('Y in t-SNE')
plt.legend(loc='upper left')
plt.title('t-SNE visualization of test data')
plt.show()

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import json
import random
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

In [ ]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
model.to(device)

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2DoubleHeadsModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [ ]:
# Get the current vocabulary size
original_vocab_size = len(tokenizer)

# Add the new words to the tokenizer

# Check if each new word is already in the GPT2 vocabulary

vocab = tokenizer.get_vocab()


num_new_words = len(words_test)

num_added = tokenizer.add_tokens([i.lower() for i in words_test])
model.resize_token_embeddings(len(tokenizer))

#Compute the distribution from which we’ll sample:
params = model.state_dict()
embeddings = params['transformer.wte.weight']

#load in our new embeddings into the model:
new_embeddings = torch.squeeze(torch.stack(test, dim=0), 1)
embeddings[-num_new_words:,:] = new_embeddings
params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
model.load_state_dict(params)


<All keys matched successfully>

In [ ]:
model.save_pretrained("gpt2_predict_model_30")
tokenizer.save_pretrained("gpt2_predict_model_30/tokenizer.json")

('gpt2_predict_model_30/tokenizer.json/tokenizer_config.json',
 'gpt2_predict_model_30/tokenizer.json/special_tokens_map.json',
 'gpt2_predict_model_30/tokenizer.json/vocab.json',
 'gpt2_predict_model_30/tokenizer.json/merges.txt',
 'gpt2_predict_model_30/tokenizer.json/added_tokens.json',
 'gpt2_predict_model_30/tokenizer.json/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

model = GPT2DoubleHeadsModel.from_pretrained('gpt2_predict_model')
model = model.to(device)

tokenizer = AutoTokenizer.from_pretrained('gpt2_predict_model/tokenizer.json')

In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [2, 5, 10, 25]

for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

NUM_CHOICES: 2
On iteration 1000, accuracy is 0.523 equal would be 0.5 count is 1000.0 [0.523, 0.477]
On iteration 2000, accuracy is 0.5095190380761523 equal would be 0.5 count is 1996.0 [0.5095190380761523, 0.4904809619238477]
On iteration 3000, accuracy is 0.5135225375626044 equal would be 0.5 count is 2995.0 [0.5135225375626044, 0.48647746243739565]
On iteration 4000, accuracy is 0.5102705410821643 equal would be 0.5 count is 3992.0 [0.5102705410821643, 0.48972945891783565]
On iteration 5000, accuracy is 0.5109196553796834 equal would be 0.5 count is 4991.0 [0.5109196553796834, 0.4890803446203166]
On iteration 6000, accuracy is 0.5118530884808014 equal would be 0.5 count is 5990.0 [0.5118530884808014, 0.48814691151919865]
On iteration 7000, accuracy is 0.5110888539132923 equal would be 0.5 count is 6989.0 [0.5110888539132923, 0.48891114608670766]
On iteration 8000, accuracy is 0.5115172759138709 equal would be 0.5 count is 7988.0 [0.5115172759138709, 0.4884827240861292]
On iteration

In [ ]:
def accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
MSE_loss = torch.nn.MSELoss()

print(predict_y.shape)
t = b_labels[:, np.newaxis, :]
print(t.shape)

MSE_loss(predict_y, t)


torch.Size([1, 768])


NameError: ignored

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

bert = BertForMaskedLM.from_pretrained('bert-base-uncased',return_dict = True)

In [ ]:
# print(words)
# for i in words:
#   print(i)
f = open("words.txt")
already_knew = 0
total = 0
for i in f.readlines():
  word = i.strip()
  if word in words or "Ġ"+word in words:
    # print("already knew", i)
    already_knew += 1
  total += 1
print(already_knew, total)

In [ ]:
print(tokenizer.tokenize("hi bbbbb"))
# text_index = tokenizer.encode('man')
new_word_list = ["bbbbb"]

num_new_words = len(new_word_list)
#print(new_word_list[0])
#print(tokenizer.convert_ids_to_tokens(tokenizer((new_word_list[0]))['input_ids']))
num_added = tokenizer.add_tokens(new_word_list)
model.resize_token_embeddings(len(tokenizer))

print(tokenizer.tokenize("hi bbbb"))

# vector1 = model.transformer.wte.weight[words["Love"],:]
# vector2 = model.transformer.wte.weight[words["love"],:]

# print(vector1 - vector2)